# 🔄 Validación Cruzada (Cross Validation, CV)

La **validación cruzada** es una técnica para evaluar el rendimiento de un modelo y ajustar sus hiperparámetros de manera más robusta.  
Incluso cuando se usa validación cruzada, se debe reservar un **conjunto de prueba independiente** para la evaluación final.  

---

## 📌 Método Hold-out (Retención)

### Procedimiento:
1. Dividir el conjunto de datos en **entrenamiento** y **prueba**.  
2. Usar el conjunto de entrenamiento para entrenar el modelo.  
3. Usar el conjunto de prueba para estimar el rendimiento de generalización.  

### Variación:
- Dividir el conjunto de entrenamiento en dos partes:  
  - **Entrenamiento** → para ajustar distintos modelos.  
  - **Validación** → para comparar configuraciones de parámetros y seleccionar el mejor modelo.  

👉 Este proceso se conoce como **selección de modelo (model selection)**.  
Su objetivo es encontrar los **valores óptimos de los hiperparámetros**.  

---

## 📌 k-fold Cross Validation (Validación cruzada k-fold)

### Procedimiento básico:
1. Dividir el conjunto de entrenamiento en **k subconjuntos (folds)**, sin reemplazo.  
2. Para cada fold:  
   - Entrenar el modelo con **k – 1 folds**.  
   - Evaluar el modelo en el **fold restante**.  
3. Repetir el proceso **k veces**, cambiando el fold de evaluación en cada iteración.  
4. Calcular la **puntuación final** como el promedio de las k mediciones.  

### Resultados:
- Se obtienen **k modelos** y **k estimaciones de rendimiento**.  
- La media de las puntuaciones es una estimación más robusta y menos sensible a cómo se partió el conjunto de datos (comparado con hold-out).  

### Usos:
- Ajustar el modelo.  
- Seleccionar hiperparámetros óptimos.  
- Reentrenar el modelo final con **todos los datos de entrenamiento**.  
- Evaluar finalmente con el **conjunto de prueba independiente**.  

📌 Valor común: `k = 10`.  
- Para conjuntos pequeños, conviene usar un `k` más alto.  

---

## 📌 Validación Cruzada k-fold Estratificada (Stratified k-fold)

- Es una **variación** de la validación cruzada k-fold.  
- Asegura que la **proporción de clases** se mantenga en cada fold.  
- Proporciona **mejores estimaciones de sesgo y varianza**, especialmente cuando las clases están **desbalanceadas**.  

---

## 📊 En resumen

- **Hold-out** → simple y rápido, pero depende mucho de cómo se dividen los datos.  
- **k-fold CV** → más robusto, proporciona un promedio de rendimiento.  
- **Stratified k-fold** → mejor opción cuando hay **clases desbalanceadas**.  


---
---
# ILUSTRACION

## Ilustración

### Validación cruzada: evaluando el rendimiento del estimador

Adaptado de [scikit learn] (Validación cruzada: evaluando el rendimiento del estimador)

Aprender los parámetros de una función de predicción y probarla en los mismos datos es un error metodológico:  
un modelo que simplemente repetiría las etiquetas de las muestras que acaba de ver tendría una puntuación perfecta,  
pero fallaría al predecir cualquier cosa en datos aún no vistos.  
A esta situación se le llama **sobreajuste (overfitting)**.

Para evitarlo, es una práctica común al realizar un experimento de aprendizaje automático (supervisado) reservar parte  
de los datos disponibles como un **conjunto de prueba** `X_test`, `y_test`.  

Nota: la palabra “experimento” no debe entenderse como de uso académico exclusivo; incluso en entornos comerciales,  
el aprendizaje automático normalmente comienza de manera experimental.

En *scikit-learn*, una división aleatoria en conjuntos de entrenamiento y prueba se puede calcular rápidamente con la función auxiliar  
`train_test_split`. Carguemos el conjunto de datos *iris* para ajustar una máquina de vectores de soporte lineal sobre él.


In [10]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

# Asi se cargaria un dataset de sns pero boston ya no esta disponible
# boston = datasets.load_boston()
boston = pd.read_csv("/media/sf_MV_COMP/data/housing.data", sep="\s+", header = None)
boston.head()

<>:14: SyntaxWarning: invalid escape sequence '\s'
<>:14: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_5784/4186260902.py:14: SyntaxWarning: invalid escape sequence '\s'
  boston = pd.read_csv("/media/sf_MV_COMP/data/housing.data", sep="\s+", header = None)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [11]:
import sys
import sklearn

## ¿Por qué separar en `X` e `y`?

Cuando cargamos el dataset con `pandas.read_csv`, todas las columnas quedan juntas en un **DataFrame**.  
Sin embargo, para entrenar un modelo en **scikit-learn** necesitamos **separar**:

- `X` → las **características de entrada** (features), es decir, las columnas que usamos para predecir.  
- `y` → la **variable objetivo** (target), es decir, la columna que queremos que el modelo aprenda a predecir.  

### Ejemplo con el dataset Boston Housing:
- `X = boston.iloc[:, :-1]` → todas las columnas menos la última (información de las casas).  
- `y = boston.iloc[:, -1]` → la última columna (precio medio de la vivienda).

| Características (X)        | Objetivo (y)      |
|-----------------------------|-------------------|
| Habitaciones, edad, etc... | Precio de la casa |

👉 **En resumen:** separar en `X` e `y` es necesario porque el modelo aprende una función que transforma `X` en una predicción de `y`.


In [14]:
X = boston.iloc[:, :-1]
y = boston.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

print(X_train.shape, y_train.shape)

print(X_test.shape, y_test.shape)

regression = svm.SVR(kernel='linear', C=1).fit(X_train, y_train)
regression.score(X_test, y_test)

(303, 13) (303,)
(203, 13) (203,)


0.667431382173115

Cuando se evalúan diferentes configuraciones ("hiperparámetros") para estimadores, como el parámetro `C` que debe establecerse manualmente para un SVM, aún existe el riesgo de sobreajuste en el conjunto de prueba porque los parámetros pueden ajustarse hasta que el rendimiento sea óptimo.

De esta manera, el conocimiento sobre el conjunto de prueba puede "filtrarse" en el modelo y las métricas de evaluación ya no informan sobre el rendimiento de generalización.

Para resolver este problema, se puede reservar otra parte del conjunto de datos como un llamado "conjunto de validación": el entrenamiento procede con el conjunto de entrenamiento, después la evaluación se hace con el conjunto de validación y, cuando el experimento parece ser exitoso, la evaluación final se hace en el conjunto de prueba.

Sin embargo, al dividir los datos disponibles en tres conjuntos, reducimos drásticamente el número de muestras que se pueden usar para entrenar el modelo, y los resultados pueden depender de una elección aleatoria particular para el par de conjuntos (entrenamiento, validación).

Una solución a este problema, como se discutió antes, es un procedimiento llamado **validación cruzada** (abreviado CV). Un conjunto de prueba todavía debe reservarse para la evaluación final, pero el conjunto de validación ya no es necesario al hacer CV. En el enfoque básico, llamado **validación cruzada k-fold**, el conjunto de entrenamiento se divide en k subconjuntos más pequeños (otros enfoques se describen a continuación, pero en general siguen los mismos principios). El procedimiento seguido para cada uno de los k “folds” es:

- Un modelo se entrena usando k-1 de los folds como datos de entrenamiento.  
- El modelo resultante se valida en la parte restante de los datos (es decir, se usa como conjunto de prueba para calcular una métrica de rendimiento como la exactitud).  

La medida de rendimiento reportada por la validación cruzada k-fold es entonces el promedio de los valores calculados en el bucle. Este enfoque puede ser computacionalmente costoso, pero no desperdicia tantos datos (como ocurre al fijar arbitrariamente un conjunto de validación), lo que es una gran ventaja en problemas como la inferencia inversa, donde el número de muestras es muy pequeño.


---
## COMPUTANDO COSS-VALIDATION METRICAS

In [15]:
from sklearn.model_selection import cross_val_score
regression = svm.SVR(kernel='linear', C=1)
scores = cross_val_score(regression, X, y, cv=5)
scores

array([0.77285459, 0.72771739, 0.56131914, 0.15056451, 0.08212844])

La puntuación media y el intervalo de confianza del 95% de la estimación de la puntuación son, por lo tanto, los siguientes:

In [19]:
print("accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() ** 2))

accuracy: 0.46 (+/- 0.08)


De forma predeterminada, la puntuación calculada en cada iteración de la validación cruzada es el método `score` del estimador.  
Es posible cambiar esto utilizando el parámetro `scoring`.


In [20]:
from sklearn import metrics
scores = cross_val_score(regression, X, y, cv=5, scoring='neg_mean_squared_error')
scores

array([ -7.84451123, -24.78772444, -35.13272326, -74.50555945,
       -24.40465975])

Consulta **The scoring parameter: defining model evaluation rules** para más detalles.  
En el caso del conjunto de datos *Iris*, las muestras están equilibradas entre las clases objetivo,  
por lo tanto, la exactitud y la puntuación F1 son casi iguales.

Cuando el argumento `cv` es un número entero, `cross_val_score` usa las estrategias **KFold** o **StratifiedKFold** por defecto,  
siendo esta última utilizada si el estimador proviene de `ClassifierMixin`.


## K-Fold

### ¿Qué es?
**K-Fold Cross Validation** es una técnica de validación cruzada donde el conjunto de datos se divide en *k* subconjuntos (llamados *folds*).  
El modelo se entrena *k* veces, cada vez usando *k-1* folds para entrenamiento y el fold restante para validación.

### Procedimiento
1. Se divide el dataset en *k* folds de tamaño similar.  
2. En cada iteración:
   - Se entrena el modelo con *k-1* folds.  
   - Se valida con el fold restante.  
3. Se repite el proceso *k* veces, cambiando el fold que se deja para validación.  
4. Se calcula la métrica promedio de rendimiento.

### Ejemplo (k=5)
- Iteración 1: folds 1-4 entrenan, fold 5 valida  
- Iteración 2: folds 1-3 y 5 entrenan, fold 4 valida  
- …  
- Iteración 5: folds 2-5 entrenan, fold 1 valida  

### Ventajas
- Utiliza **todo el dataset** tanto para entrenar como para validar (en diferentes momentos).  
- Da una evaluación más robusta que un simple train/test split.

### Desventajas
- Puede ser **computacionalmente costoso**, ya que el modelo se entrena *k* veces.  



In [21]:
import numpy as np
from sklearn.model_selection import KFold

X = ["a","b","c","d"]
kf = KFold(n_splits = 2)
for train, test in kf.split(X):
    print("%s %s" % (train, test))

[2 3] [0 1]
[0 1] [2 3]


## Stratified K-Fold

### ¿Qué es?
**Stratified K-Fold** es una variante del método de validación cruzada **K-Fold** que se utiliza principalmente en **problemas de clasificación**.

### Diferencia con K-Fold normal
- **K-Fold normal**: divide el dataset en *k* subconjuntos (*folds*) de manera aleatoria, intentando que tengan un tamaño similar.  
- **Stratified K-Fold**: además de dividir en *k* folds, **mantiene la misma proporción de clases en cada fold** que en el conjunto de datos original.

### Ejemplo
Dataset con 100 muestras:
- 80 de la clase A  
- 20 de la clase B  

- Con **K-Fold normal**, algunos *folds* podrían tener muy pocos o incluso ningún ejemplo de la clase B.  
- Con **Stratified K-Fold**, cada fold tendrá aproximadamente un **80% A y 20% B**, igual que el dataset completo.

### ¿Por qué es importante?
- En datasets **desbalanceados**, el K-Fold normal puede generar evaluaciones poco realistas.  
- **Stratified K-Fold** asegura que el modelo se entrene y valide siempre con una representación equilibrada de las clases.  

### En Scikit-learn
Cuando se usa `cross_val_score` con clasificadores (`ClassifierMixin`), por defecto se aplica **StratifiedKFold** en lugar de K-Fold.



In [22]:
from sklearn.model_selection import StratifiedKFold

X = np.ones(10)
y = [0,0,0,0,1,1,1,1,1,1]
skf = StratifiedKFold(n_splits = 3)
for train, test in skf.split(X, y):
    print("%s %s" % (train, test))

[2 3 6 7 8 9] [0 1 4 5]
[0 1 3 4 5 8 9] [2 6 7]
[0 1 2 4 5 6 7] [3 8 9]


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.pipeline import make_pipeline

# pipe_lr = make_pipeline(StandardScaler(), PCA(n_components = 2), LogisticRegression(random_state = 1))
pipe_svm = make_pipeline(StandardScaler(), PCA(n_components = 2), svm.SVR(kernel = 'linear', C = 1))

pipe_svm.fit(X_train, y_train)
y_pred = pipe_svm.predict(X_test)
print('Test accuracy: %.3f' % pipe_svm.score(X_test, y_test))

Test accuracy: 0.391


## Pipeline con PCA y SVM (Scikit-learn)

### ¿Qué hace este código?
1. **Escala los datos** con `StandardScaler` (media 0, varianza 1).  
2. **Reduce la dimensionalidad** con `PCA(n_components=2)`.  
3. **Entrena un modelo SVM de regresión** (`SVR`) con kernel lineal y `C=1`.  
4. **Evalúa el rendimiento** en un conjunto de prueba.



In [25]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(estimator = pipe_svm, X = X_train, y = y_train, cv = 10, n_jobs = 1)
print('CV accuracy scores: %s' % scores)

CV accuracy scores: [0.63971176 0.43579197 0.46977821 0.25027246 0.5124364  0.26221374
 0.30877195 0.54528563 0.37810066 0.47313549]


## Validación cruzada con cross_val_score

### Descripción
Este fragmento aplica **validación cruzada** usando la función `cross_val_score` de Scikit-learn.  
El objetivo es evaluar el rendimiento del modelo de manera más robusta que con un simple train/test split.

### Paso a paso
1. **Importación**  
   Se utiliza la función `cross_val_score` para automatizar la validación cruzada.

2. **Ejecución**  
   - El modelo `pipe_svm` se entrena y valida mediante **10 folds** (particiones).  
   - En cada iteración, el modelo se entrena con 9 folds y se valida con el fold restante.  
   - El resultado es una lista con las métricas obtenidas en cada fold.

3. **Parámetros principales**  
   - **estimator = pipe_svm** → el pipeline con escalado, PCA y SVM.  
   - **X = X_train, y = y_train** → datos de entrenamiento y sus etiquetas.  
   - **cv = 10** → número de folds en la validación cruzada.  
   - **n_jobs = 1** → número de núcleos de CPU utilizados (1 = secuencial).

4. **Salida**  
   - Se imprimen los puntajes de cada fold (precisión o R² dependiendo del tipo de modelo).  
   - Estos valores permiten analizar la variabilidad del modelo y detectar si es estable o inestable.


In [26]:
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy: 0.428 +/- 0.121


## Promedio y desviación estándar de la validación cruzada

### Descripción
Este comando muestra un **resumen del rendimiento del modelo** tras la validación cruzada.

### Explicación
- **np.mean(scores)** → calcula el promedio de las métricas obtenidas en todos los folds.  
  - Representa el **rendimiento medio** del modelo.  
- **np.std(scores)** → calcula la desviación estándar de esas métricas.  
  - Indica la **variabilidad o estabilidad** del modelo entre los distintos folds.  

### Salida
El resultado se imprime en el formato:  
**CV accuracy: valor_promedio +/- desviación**  

Ejemplo:  
`CV accuracy: 0.428 +/- 0.121`  
- **0.428** → rendimiento medio del modelo.  
- **0.121** → dispersión de los resultados; cuanto menor sea, más estable es el modelo.
